In [1]:
pip install git+https://github.com/egoolish/ecp_python

  Cloning https://github.com/egoolish/ecp_python to /private/var/folders/1r/_mt76hv974ddmv82vjxl18pm0000gn/T/pip-req-build-x86spb94
  Stored in directory: /private/var/folders/1r/_mt76hv974ddmv82vjxl18pm0000gn/T/pip-ephem-wheel-cache-dt1nij_c/wheels/16/c4/81/5f49ffa704964ad0a2def2f6626cc7e67a718ffa2355d93782
Successfully built ecp
Note: you may need to restart the kernel to use updated packages.


In [2]:
#import needed packages

import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
import time
from ecp import e_divisive, e_agglomerative
%matplotlib inline

In [3]:
# P is the (k x k) transition matrix
# S is the vector of averages for each state (of length k)
# sigma is the shared variance
# n is the number of points to generate (length of time series)
# if debug = True, return state_gen rather than time_series.
def mkv_to_ts(P, S, sigma = 1, n = 10000, debug=False):
    k = len(S)
    base_matrix = np.tile(S, (n, 1)).T
    noise_matrix = np.random.normal(0, sigma, (k, n))
    full_matrix = base_matrix + noise_matrix
    
    state_gen = np.empty(n, dtype = np.int32)
    state_gen[0] = 0
    for i in range(1, n):
        state_gen[i] = np.random.choice(k, p=P[state_gen[i-1]])
    return state_gen if debug else np.choose(state_gen, full_matrix)

In [4]:
def MCTS(ts, K, ms, num_of_states, alpha=0):
    sol_dict = e_divisive.e_divisive(X = np.reshape(ts, (len(ts), 1)), k=K, alpha=2, min_size=ms)

    clusters = sol_dict['cluster']
    num_of_clusters = clusters[-1]+1

    find_means = np.vectorize(lambda x : np.mean(ts[clusters == x]))
    cluster_means = find_means(np.arange(num_of_clusters))

    sort_c_idx = np.argsort(cluster_means).argsort()
    sort_c = np.sort(cluster_means)

    sol_dict_2 = e_divisive.e_divisive(X = np.reshape(sort_c, (len(sort_c), 1)), k=num_of_states-1, alpha=2, min_size=2)

    cluster_labels = np.zeros(num_of_clusters, dtype=np.int32)
    ests = sol_dict_2['estimates']
    for i in range(1, len(ests)):
        cluster_labels[(sort_c_idx >= ests[i-1]) & (sort_c_idx < ests[i])] = i - 1

    new_ests = [sol_dict['estimates'][0]]
    new_labs = [cluster_labels[0]]
    for i in range(1, len(cluster_labels)):
        if cluster_labels[i] != cluster_labels[i-1]:
            new_ests.append(sol_dict['estimates'][i])
            new_labs.append(cluster_labels[i])
    new_ests.append(sol_dict['estimates'][-1])
    final_ests = np.array(new_ests)
    new_labs = np.array(new_labs)

    lens = np.diff(final_ests)
    find_avg_time = np.vectorize(lambda x : np.mean(lens[new_labs == x]))
    avg_times = find_avg_time(np.arange(num_of_states))

    P_diag = np.vectorize(lambda x : (x - 1)/x)(avg_times)

    est_P = np.zeros((num_of_states, num_of_states)) + alpha
    for i in range(1, len(cluster_labels)):
        est_P[cluster_labels[i-1], cluster_labels[i]] += 1
    np.fill_diagonal(est_P, 0)
    est_P = normalize(est_P, axis = 1, norm='l1')
    est_P = (est_P.T*(1 - P_diag)).T
    np.fill_diagonal(est_P, P_diag)

    final_clusters = np.repeat(np.arange(0,len(np.diff(final_ests))), np.diff(final_ests))
    find_seg_sums = np.vectorize(lambda x : np.sum(ts[final_clusters == x]))
    seg_sums = find_seg_sums(np.arange(final_clusters[-1]+1))

    find_time = np.vectorize(lambda x: np.sum(lens[new_labs == x]))
    total_times = find_time(np.arange(num_of_states))

    find_state_sums = np.vectorize(lambda x: np.sum(seg_sums[new_labs == x]))
    est_S = find_state_sums(np.arange(num_of_states))/total_times

    return est_P, est_S

In [5]:
def MCTS_agglo(ts, K, ms, num_of_states, alpha=0):
    sol_dict = e_divisive.e_divisive(X = np.reshape(ts, (len(ts), 1)), k=K, alpha=2, min_size=ms)

    clusters = sol_dict['cluster']
    uniq_vals = np.unique(clusters)
    num_of_clusters = len(uniq_vals)

    find_means = np.vectorize(lambda x : np.mean(ts[clusters == x]))
    cluster_means = find_means(uniq_vals)

    sort_c_idx = np.argsort(cluster_means).argsort()
    sort_c = np.sort(cluster_means)

    sol_dict_2 = e_agglomerative.e_agglo(X = np.reshape(sort_c, (len(sort_c), 1)), alpha=2, member=np.arange(np.reshape(sort_c, (len(sort_c), 1)).shape[0]))

    cluster_labels = np.zeros(num_of_clusters, dtype=np.int32)
    ests = sol_dict_2['estimates']
    for i in range(1, len(ests)):
        cluster_labels[(sort_c_idx >= ests[i-1]) & (sort_c_idx < ests[i])] = i - 1

    new_ests = [sol_dict['estimates'][0]]
    new_labs = [cluster_labels[0]]
    for i in range(1, len(cluster_labels)):
        if cluster_labels[i] != cluster_labels[i-1]:
            new_ests.append(sol_dict['estimates'][i])
            new_labs.append(cluster_labels[i])
    new_ests.append(sol_dict['estimates'][-1])
    final_ests = np.array(new_ests)
    new_labs = np.array(new_labs)

    lens = np.diff(final_ests)
    find_avg_time = np.vectorize(lambda x : np.mean(lens[new_labs == x]))
    avg_times = find_avg_time(np.arange(num_of_states))

    P_diag = np.vectorize(lambda x : (x - 1)/x)(avg_times)

    est_P = np.zeros((num_of_states, num_of_states)) + alpha
    for i in range(1, len(cluster_labels)):
        est_P[cluster_labels[i-1], cluster_labels[i]] += 1
    np.fill_diagonal(est_P, 0)
    est_P = normalize(est_P, axis = 1, norm='l1')
    est_P = (est_P.T*(1 - P_diag)).T
    np.fill_diagonal(est_P, P_diag)

    final_clusters = np.repeat(np.arange(0,len(np.diff(final_ests))), np.diff(final_ests))
    find_seg_sums = np.vectorize(lambda x : np.sum(ts[final_clusters == x]))
    seg_sums = find_seg_sums(np.arange(final_clusters[-1]+1))

    find_time = np.vectorize(lambda x: np.sum(lens[new_labs == x]))
    total_times = find_time(np.arange(num_of_states))

    find_state_sums = np.vectorize(lambda x: np.sum(seg_sums[new_labs == x]))
    est_S = find_state_sums(np.arange(num_of_states))/total_times

    return est_P, est_S

In [7]:
P = np.array([[0.80, 0.10, 0.10],
              [0.05, 0.90, 0.05],
              [0.04, 0.04, 0.92]])
S = np.array([-5, 0, 5])

N = 1000
K = int(N/(1/(1 - P[0, 0]))) + N/100
ms = 2
num_of_states = len(S)

t1 = time.time()
ts = mkv_to_ts(P, S, n=N)
t2 = time.time()

est_P, est_S = MCTS(ts, K, ms, num_of_states)
t3 = time.time()

print("Time Series Length: " + str(N))
print("Generation took: " + str(t2 - t1))
print("MCTS took: " + str(t3 - t2))
print("np.linalg.norm(P - est_P): " + str(np.linalg.norm(P - est_P)))
print("np.linalg.norm(S - est_S): " + str(np.linalg.norm(S - est_S)))
print(est_S)
print(est_P)
print()

Time Series Length: 1000
Generation took: 0.05111503601074219
MCTS took: 12.890102624893188
np.linalg.norm(P - est_P): 0.10131514730767167
np.linalg.norm(S - est_S): 0.39817609476059745
[-4.91493299 -0.11238274  4.62760515]
[[0.85915493 0.04694836 0.09389671]
 [0.05054557 0.86184211 0.08761232]
 [0.02898551 0.06625259 0.9047619 ]]



In [15]:
# P = np.array([[0.90, 0.10, 0.00],
#               [0.005, 0.99, 0.005],
#               [0.01, 0.04, 0.95]])
# S = np.array([-5, 0, 5])

# N = 1000
# K = int(N/(1/(1 - P[0, 0]))) + N/100
# ms = 2
# num_of_states = len(S)

t1 = time.time()
# ts = mkv_to_ts(P, S, n=N)
t2 = time.time()

est_P, est_S = MCTS_agglo(ts, K, ms, num_of_states)
t3 = time.time()

print("Time Series Length: " + str(N))
print("Generation took: " + str(t2 - t1))
print("MCTS took: " + str(t3 - t2))
print("np.linalg.norm(P - est_P): " + str(np.linalg.norm(P - est_P)))
print("np.linalg.norm(S - est_S): " + str(np.linalg.norm(S - est_S)))
print(est_S)
print(est_P)
print()

Time Series Length: 5000
Generation took: 3.790855407714844e-05
MCTS took: 215.46539902687073
np.linalg.norm(P - est_P): 0.0666839864548222
np.linalg.norm(S - est_S): 0.3699216149705774
[-4.95967022  0.36325992  5.0570766 ]
[[0.84607438 0.0857438  0.06818182]
 [0.03990128 0.89927361 0.06082512]
 [0.03355363 0.06354855 0.90289781]]



In [ ]:
print(S)
print(est_S)
print(P)
print(est_P)

In [28]:
print(S)
print(est_S)
print(P)
print(est_P)

[ 5  0 -5]
[-4.98227602 -0.05093479  4.80078408]
[[0.9   0.1   0.   ]
 [0.005 0.99  0.005]
 [0.01  0.04  0.95 ]]
[[0.96774194 0.03225806 0.        ]
 [0.00865513 0.98268975 0.00865513]
 [0.         0.11111111 0.88888889]]


# Testing fixed P, S, but varying length of TS:

In [16]:
P = np.array([[0.5, 0.25, 0.25],
              [0.2, 0.7, 0.1],
              [0.2, 0.4, 0.4]])
S = np.array([-5, 0, 5])


Ns = [500, 1000, 2000, 5000]

for N in Ns:
    K = int(N/(1/(1 - P[0, 0]))) + N/100
    ms = 2
    num_of_states = len(S)

    t1 = time.time()
    ts = mkv_to_ts(P, S, n=N)
    t2 = time.time()

    est_P, est_S = MCTS(ts, K, ms, num_of_states)
    t3 = time.time()

    print("Time Series Length: " + str(N))
    print("Generation took: " + str(t2 - t1))
    print("MCTS took: " + str(t3 - t2))
    print("np.linalg.norm(P - est_P): " + str(np.linalg.norm(P - est_P)))
    print("np.linalg.norm(S - est_S): " + str(np.linalg.norm(S - est_S)))
    print(est_S)
    print(est_P)
    print()

Time Series Length: 500
Generation took: 0.03162789344787598
MCTS took: 2.744338035583496
np.linalg.norm(P - est_P): 0.4317264905396882
np.linalg.norm(S - est_S): 1.310411681880927
[-3.82502453  0.31856108  4.51509768]
[[0.71005917 0.17751479 0.11242604]
 [0.15454545 0.77922078 0.06623377]
 [0.13       0.21       0.66      ]]

Time Series Length: 1000
Generation took: 0.034195899963378906
MCTS took: 7.512833118438721
np.linalg.norm(P - est_P): 0.4309092559904749
np.linalg.norm(S - est_S): 1.332695422874533
[-4.14615284  0.09678472  3.98134656]
[[0.72704082 0.16071429 0.1122449 ]
 [0.17233742 0.77336449 0.05429809]
 [0.18888889 0.18333333 0.62777778]]

Time Series Length: 2000
Generation took: 0.06187319755554199
MCTS took: 53.05824279785156
np.linalg.norm(P - est_P): 0.39204503214478853
np.linalg.norm(S - est_S): 1.339438929828084
[-3.91162088  0.29491956  4.27712381]
[[0.69142857 0.18657807 0.12199336]
 [0.16504854 0.77562028 0.05933118]
 [0.16621984 0.20911528 0.62466488]]

Time Seri

# Testing larger number of states

In [9]:
P = np.array([[0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81]])
S = np.array([-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18])

N = 5000
K = int(N/(1/(1 - P[0, 0]))) + N/100
ms = 2
num_of_states = len(S)

t1 = time.time()
ts = mkv_to_ts(P, S, n=N)
t2 = time.time()

est_P, est_S = MCTS(ts, K, ms, num_of_states)
t3 = time.time()

print("Time Series Length: " + str(N))
print("Generation took: " + str(t2 - t1))
print("MCTS took: " + str(t3 - t2))
print("np.linalg.norm(P - est_P): " + str(np.linalg.norm(P - est_P)))
print("np.linalg.norm(S - est_S): " + str(np.linalg.norm(S - est_S)))
print(est_S)
print(est_P)
print()

Time Series Length: 5000
Generation took: 0.1999492645263672
MCTS took: 177.06409788131714
np.linalg.norm(P - est_P): 0.1784732302451087
np.linalg.norm(S - est_S): 6.095032405280327
[-19.79852843 -17.84095643 -16.36616391 -15.59577316 -14.07197905
 -12.10193418  -9.92987929  -8.29067122  -6.52037646  -4.11990908
  -1.78377104   0.89113157   3.08207512   5.80832021   8.08928627
  10.61000779  12.34374068  13.89893103  15.72806037  17.9743032 ]
[[0.83895131 0.01123596 0.00374532 0.00749064 0.00374532 0.00374532
  0.00374532 0.00749064 0.00374532 0.00749064 0.01498127 0.00749064
  0.01498127 0.00749064 0.00749064 0.01123596 0.00374532 0.01498127
  0.01872659 0.00749064]
 [0.01869159 0.80841121 0.00934579 0.00934579 0.0046729  0.0046729
  0.         0.0046729  0.01869159 0.01401869 0.0046729  0.02336449
  0.         0.00934579 0.01869159 0.01401869 0.         0.00934579
  0.01869159 0.00934579]
 [0.00840336 0.03361345 0.75630252 0.00840336 0.00840336 0.00840336
  0.03361345 0.         0.  

# Michael's Method

In [33]:
P = np.array([[0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81, 0.01],
              [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.81]])
S = np.array([-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18])



Ns = [500, 5000]

for N in Ns:
    K = int(N/(1/(1 - P[0, 0]))) + N/100
    ms = 2
    num_of_states = len(S)

    t1 = time.time()
    ts = mkv_to_ts(P, S, n=N)
    t2 = time.time()

    est_P, est_S = MCTS(ts, K, ms, num_of_states, alpha=1/num_of_states)
    t3 = time.time()

    print("Time Series Length: " + str(N))
    print("Generation took: " + str(t2 - t1))
    print("MCTS took: " + str(t3 - t2))
    print("np.linalg.norm(P - est_P): " + str(np.linalg.norm(P - est_P)))
    print("np.linalg.norm(S - est_S): " + str(np.linalg.norm(S - est_S)))
    print(est_S)
    print(est_P)
    print()

Time Series Length: 500
Generation took: 0.021036148071289062
MCTS took: 1.8718409538269043
np.linalg.norm(P - est_P): 0.6290232420333294
np.linalg.norm(S - est_S): 5.269492334810537
[-21.07453973 -20.02048677 -17.76721108 -14.42850532 -12.80081365
  -9.94392796  -7.88305426  -5.6344462   -4.02718159  -2.65326372
   1.20391307   2.46125287   3.87926391   5.03453122   6.47027558
   8.2842243   10.10117615  11.45325321  15.85871902  17.97931658]
[[0.7        0.07974684 0.00379747 0.00379747 0.00379747 0.00379747
  0.00379747 0.00379747 0.00379747 0.00379747 0.00379747 0.00379747
  0.00379747 0.00379747 0.00379747 0.00379747 0.07974684 0.07974684
  0.00379747 0.00379747]
 [0.00127324 0.84848485 0.00127324 0.00127324 0.00127324 0.00127324
  0.00127324 0.02673797 0.00127324 0.00127324 0.02673797 0.00127324
  0.00127324 0.00127324 0.00127324 0.02673797 0.00127324 0.00127324
  0.0522027  0.00127324]
 [0.00165107 0.00165107 0.86956522 0.00165107 0.00165107 0.00165107
  0.00165107 0.03467254 0.